# NOTE
### *This is a rough notebook, showcasing all the work done before to finalize the working code.*

In [ ]:
import requests
from bs4 import BeautifulSoup


def scrape_linkedin(search_query):
    url = f"https://www.linkedin.com/search/results/people/?keywords={search_query}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        names = soup.find_all('span', class_='name')
        job_titles = soup.find_all('div', class_='job-title')
        leads = []

        for name, title in zip(names, job_titles):
            leads.append({
                "name": name.text.strip(),
                "title": title.text.strip(),
            })

        return leads
    else:
        return []


search_query = "Director of Toxicology"
leads = scrape_linkedin(search_query)
print(leads)


In [ ]:
import requests
from bs4 import BeautifulSoup
from Bio import Entrez
import re


Entrez.email = ""


def scrape_pubmed_and_conferences(job_titles, publication_keywords, conference_urls, max_results=20):
    leads = []


    query = " AND ".join(publication_keywords)

    handle = Entrez.esearch(
        db="pubmed",
        term=query,
        retmax=max_results
    )
    record = Entrez.read(handle)
    pubmed_ids = record["IdList"]

    if pubmed_ids:
        fetch = Entrez.efetch(
            db="pubmed",
            id=",".join(pubmed_ids),
            rettype="medline",
            retmode="text"
        )

        articles = fetch.read().split("\n\n")

        for article in articles:
            author_match = re.search(r"AU  - (.+)", article)
            title_match = re.search(r"TI  - (.+)", article)
            aff_match = re.search(r"AD  - (.+)", article)

            if author_match and title_match:
                affiliation = aff_match.group(1) if aff_match else ""


                if any(jt.lower() in affiliation.lower() for jt in job_titles):
                    leads.append({
                        "name": author_match.group(1),
                        "title": next((jt for jt in job_titles if jt.lower() in affiliation.lower()), "Researcher"),
                        "publication_or_event": title_match.group(1),
                        "source": "pubmed",
                        "url": "https://pubmed.ncbi.nlm.nih.gov/"
                    })


    for url in conference_urls:
        try:
            response = requests.get(url, timeout=10)
            if response.status_code != 200:
                continue

            soup = BeautifulSoup(response.text, "html.parser")


            people_blocks = soup.find_all(
                ["div", "li", "section"],
                class_=re.compile("speaker|author|presenter|faculty", re.I)
            )

            for block in people_blocks:
                text = block.get_text(" ", strip=True)

                if any(jt.lower() in text.lower() for jt in job_titles):
                    leads.append({
                        "name": text.split(",")[0],
                        "title": next((jt for jt in job_titles if jt.lower() in text.lower()), ""),
                        "publication_or_event": soup.title.text if soup.title else "",
                        "source": "conference",
                        "url": url
                    })

        except Exception as e:
            print(f"Failed to scrape {url}: {e}")

    return leads






# job_titles = [
#     "Director",
#     "Professor",
#     "Principal Scientist"
# ]

# publication_keywords = [
#     "toxicology",
#     "drug safety"
# ]

# conference_urls = [
#
# ]

# leads = scrape_pubmed_and_conferences(
#     job_titles,
#     publication_keywords,
#     conference_urls
# )

# print(leads)


In [ ]:
import requests

HUNTER_API_KEY = "your_hunter_api_key"

def enrich_data(email_domain):
    url = f"https://api.hunter.io/v2/domain-search?domain={email_domain}&api_key={HUNTER_API_KEY}"
    response = requests.get(url)
    data = response.json()

    if 'data' in data:
        emails = [entry['value'] for entry in data['data']['emails']]
        return emails
    else:
        return []

email_domain = "pfizer.com"
emails = enrich_data(email_domain)
print(emails)


In [ ]:
def rank_leads(leads):
    for lead in leads:
        score = 0

        if 'Toxicology' in lead['title'] or 'Safety' in lead['title']:
            score += 30

        if lead.get('company_funding') == "Series B":
            score += 20

        if lead.get('published_paper'):
            score += 40

        lead['score'] = score

    sorted_leads = sorted(leads, key=lambda x: x['score'], reverse=True)
    return sorted_leads


In [ ]:
import streamlit as st
import pandas as pd

def display_dashboard(leads):
    df = pd.DataFrame(leads)
    st.title("Lead Generation Dashboard")


    st.write("### Ranked Leads")
    st.dataframe(df)


    search_query = st.text_input("Search for a lead")
    if search_query:
        filtered_leads = df[df['name'].str.contains(search_query, case=False)]
        st.write(filtered_leads)

leads_data = [
    {"name": "Dr. Jane Smith", "title": "Director of Toxicology", "company": "Pfizer", "location": "Boston", "score": 70, "company_funding": "Series B", "published_paper": True},
    {"name": "Dr. John Doe", "title": "Junior Scientist", "company": "XYZ Biotech", "location": "Remote", "score": 30, "company_funding": "Seed", "published_paper": False},
]

ranked_leads = rank_leads(leads_data)

display_dashboard(ranked_leads)


Proper Ver

In [ ]:
import requests
from bs4 import BeautifulSoup
from Bio import Entrez
import re

Entrez.email = "your_email@example.com"


def scrape_all_sources(search_query, job_titles, publication_keywords, conference_urls, max_results=20):
    leads = []

    linkedin_url = f"https://www.linkedin.com/search/results/people/?keywords={search_query}"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    try:
        response = requests.get(linkedin_url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")

            names = soup.find_all("span", class_="name")
            titles = soup.find_all("div", class_="job-title")

            for name, title in zip(names, titles):
                leads.append({
                    "name": name.text.strip(),
                    "title": title.text.strip(),
                    "company": "",
                    "source": "linkedin",
                    "publication": None,
                    "email": None,
                    "phone": None
                })
    except Exception:
        pass


    query = " AND ".join(publication_keywords)
    handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
    record = Entrez.read(handle)

    if record["IdList"]:
        fetch = Entrez.efetch(
            db="pubmed",
            id=",".join(record["IdList"]),
            rettype="medline",
            retmode="text"
        )

        articles = fetch.read().split("\n\n")

        for article in articles:
            author = re.search(r"AU  - (.+)", article)
            title = re.search(r"TI  - (.+)", article)
            affiliation = re.search(r"AD  - (.+)", article)

            if not author or not title:
                continue

            aff_text = affiliation.group(1) if affiliation else ""

            if any(jt.lower() in aff_text.lower() for jt in job_titles):
                leads.append({
                    "name": author.group(1),
                    "title": next((jt for jt in job_titles if jt.lower() in aff_text.lower()), "Researcher"),
                    "company": aff_text,
                    "source": "pubmed",
                    "publication": title.group(1),
                    "email": None,
                    "phone": None
                })

    for url in conference_urls:
        try:
            res = requests.get(url, timeout=10)
            if res.status_code != 200:
                continue

            soup = BeautifulSoup(res.text, "html.parser")

            blocks = soup.find_all(
                ["div", "li", "section"],
                class_=re.compile("speaker|author|faculty|presenter", re.I)
            )

            for block in blocks:
                text = block.get_text(" ", strip=True)

                if any(jt.lower() in text.lower() for jt in job_titles):
                    leads.append({
                        "name": text.split(",")[0],
                        "title": next((jt for jt in job_titles if jt.lower() in text.lower()), ""),
                        "company": "",
                        "source": "conference",
                        "publication": soup.title.text if soup.title else "",
                        "email": None,
                        "phone": None
                    })

        except Exception:
            pass

    return leads


In [ ]:
HUNTER_API_KEY = "your_hunter_api_key"

def enrich_lead_contacts(leads):
    for lead in leads:
        company_text = lead.get("company", "")
        domain_match = re.search(r'([a-zA-Z0-9.-]+\.[a-zA-Z]{2,})', company_text)

        if not domain_match:
            continue

        domain = domain_match.group(1)
        url = f"https://api.hunter.io/v2/domain-search?domain={domain}&api_key={HUNTER_API_KEY}"
        response = requests.get(url)

        if response.status_code != 200:
            continue

        data = response.json()

        if "data" in data and data["data"]["emails"]:
            lead["email"] = data["data"]["emails"][0]["value"]
            lead["phone"] = data["data"]["emails"][0].get("phone_number")

    return leads


In [ ]:
def rank_leads(leads):
    for lead in leads:
        score = 0

        if lead["title"] and "toxicology" in lead["title"].lower():
            score += 30

        if lead["email"]:
            score += 25

        if lead["source"] == "pubmed":
            score += 25

        if lead["source"] == "conference":
            score += 20

        lead["score"] = score

    return sorted(leads, key=lambda x: x["score"], reverse=True)


In [ ]:
def run_lead_pipeline():
    search_query = "Director of Toxicology"
    job_titles = ["Director", "Professor", "Principal Scientist"]
    publication_keywords = ["toxicology", "drug safety"]
    conference_urls = ["https://example-conference.org/speakers"]

    leads = scrape_all_sources(
        search_query,
        job_titles,
        publication_keywords,
        conference_urls
    )


    leads = enrich_lead_contacts(leads)


    ranked_leads = rank_leads(leads)

    return ranked_leads


In [ ]:
pip install streamlit pandas


In [ ]:
#APP.PY

import streamlit as st
import pandas as pd




def display_scoreboard(ranked_leads):
    st.set_page_config(page_title="Lead Scoreboard", layout="wide")

    st.title("Lead Generation Scoreboard")

    if not ranked_leads:
        st.warning("No leads found.")
        return


    df = pd.DataFrame(ranked_leads)

    desired_columns = [
        "name",
        "title",
        "email",
        "phone",
        "company",
        "source",
        "score"
    ]

    df = df[[col for col in desired_columns if col in df.columns]]


    df = df.sort_values(by="score", ascending=False)


    search = st.text_input("Search by name, title, or company")

    if search:
        df = df[df.apply(
            lambda row: search.lower() in row.astype(str).str.lower().to_string(),
            axis=1
        )]


    st.subheader("Ranked Leads")
    st.dataframe(
        df,
        use_container_width=True,
        hide_index=True
    )


    csv = df.to_csv(index=False).encode("utf-8")
    st.download_button(
        label="Download Scoreboard as CSV",
        data=csv,
        file_name="ranked_leads_scoreboard.csv",
        mime="text/csv"
    )




from backend_pipeline import run_lead_pipeline

ranked_leads = run_lead_pipeline()

display_scoreboard(ranked_leads)


In [ ]:
run the app: streamlit run app.py


Another

In [ ]:


import requests
from bs4 import BeautifulSoup
from Bio import Entrez
import re
import pandas as pd
import streamlit as st

Entrez.email = "your_email@example.com"
HUNTER_API_KEY = "your_hunter_api_key"


def scrape_all_sources(
    search_query,
    job_titles,
    publication_keywords,
    conference_urls,
    max_results=20
):
    leads = []


    linkedin_url = f"https://www.linkedin.com/search/results/people/?keywords={search_query}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        res = requests.get(linkedin_url, headers=headers, timeout=10)
        if res.status_code == 200:
            soup = BeautifulSoup(res.text, "html.parser")
            names = soup.find_all("span", class_="name")
            titles = soup.find_all("div", class_="job-title")
            locations = soup.find_all("span", class_="location")

            for name, title, loc in zip(names, titles, locations):
                leads.append({
                    "name": name.text.strip(),
                    "title": title.text.strip(),
                    "company": "",
                    "location": loc.text.strip(),
                    "email": None,
                    "phone": None,
                    "source": "linkedin",
                    "publication": None
                })
    except Exception:
        pass


    query = " AND ".join(publication_keywords)
    handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
    record = Entrez.read(handle)

    if record["IdList"]:
        fetch = Entrez.efetch(
            db="pubmed",
            id=",".join(record["IdList"]),
            rettype="medline",
            retmode="text"
        )
        articles = fetch.read().split("\n\n")

        for article in articles:
            author = re.search(r"AU  - (.+)", article)
            title = re.search(r"TI  - (.+)", article)
            affiliation = re.search(r"AD  - (.+)", article)

            if not author or not title:
                continue

            aff_text = affiliation.group(1) if affiliation else ""

            location_match = re.search(r',\s*([^,]+,\s*[A-Z]{2}|[^,]+,\s*[^,]+)$', aff_text)
            location = location_match.group(1) if location_match else ""

            if any(jt.lower() in aff_text.lower() for jt in job_titles):
                leads.append({
                    "name": author.group(1),
                    "title": next((jt for jt in job_titles if jt.lower() in aff_text.lower()), "Researcher"),
                    "company": aff_text,
                    "location": location,
                    "email": None,
                    "phone": None,
                    "source": "pubmed",
                    "publication": title.group(1)
                })

    for url in conference_urls:
        try:
            res = requests.get(url, timeout=10)
            if res.status_code != 200:
                continue

            soup = BeautifulSoup(res.text, "html.parser")
            blocks = soup.find_all(
                ["div", "li", "section"],
                class_=re.compile("speaker|author|faculty|presenter", re.I)
            )

            for block in blocks:
                text = block.get_text(" ", strip=True)
                if any(jt.lower() in text.lower() for jt in job_titles):
                    loc_match = re.search(r'\b[A-Z][a-z]+,\s?[A-Z]{2}\b', text)
                    leads.append({
                        "name": text.split(",")[0],
                        "title": next((jt for jt in job_titles if jt.lower() in text.lower()), ""),
                        "company": "",
                        "location": loc_match.group(0) if loc_match else "",
                        "email": None,
                        "phone": None,
                        "source": "conference",
                        "publication": soup.title.text if soup.title else ""
                    })
        except Exception:
            pass

    return leads


def enrich_lead_contacts(leads):
    for lead in leads:
        if not lead.get("company"):
            continue

        domain_match = re.search(r'([a-zA-Z0-9.-]+\.[a-zA-Z]{2,})', lead["company"])
        if not domain_match:
            continue

        domain = domain_match.group(1)
        url = f"https://api.hunter.io/v2/domain-search?domain={domain}&api_key={HUNTER_API_KEY}"

        try:
            res = requests.get(url)
            if res.status_code != 200:
                continue

            data = res.json()
            if "data" in data and data["data"]["emails"]:
                lead["email"] = data["data"]["emails"][0]["value"]
                lead["phone"] = data["data"]["emails"][0].get("phone_number")
        except Exception:
            pass

    return leads


def rank_leads(leads):
    for lead in leads:
        score = 0

        if lead["title"] and "toxicology" in lead["title"].lower():
            score += 30

        if lead["email"]:
            score += 25

        if lead["source"] == "pubmed":
            score += 25

        if lead["location"]:
            score += 10

        lead["score"] = score

    return sorted(leads, key=lambda x: x["score"], reverse=True)


def run_lead_pipeline():
    leads = scrape_all_sources(
        search_query="Director of Toxicology",
        job_titles=["Director", "Professor", "Principal Scientist"],
        publication_keywords=["toxicology", "drug safety"],
        conference_urls=["https://example-conference.org/speakers"]
    )

    leads = enrich_lead_contacts(leads)
    leads = rank_leads(leads)

    return leads

def display_scoreboard(leads):
    st.set_page_config(layout="wide")
    st.title("Lead Scoreboard")

    df = pd.DataFrame(leads).sort_values("score", ascending=False)

    columns = [
        "name",
        "title",
        "company",
        "location",
        "email",
        "phone",
        "source",
        "score"
    ]

    df = df[[c for c in columns if c in df.columns]]

    st.dataframe(df, use_container_width=True, hide_index=True)

    st.download_button(
        "Download CSV",
        df.to_csv(index=False),
        "lead_scoreboard.csv",
        "text/csv"
    )

ranked_leads = run_lead_pipeline()
display_scoreboard(ranked_leads)


another one

In [3]:
!pip install Bio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.1 MB/s eta 0:00:00


In [5]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.4 MB/s eta 0:00:00


In [6]:


import requests
from bs4 import BeautifulSoup
from Bio import Entrez
import re
import pandas as pd
import streamlit as st
from datetime import datetime



Entrez.email = "email@example.com"
HUNTER_API_KEY = "hunter_api_key"


def scrape_social_networks(job_titles):

    leads = []

    leads.append({
        "name": "Dr. Jane Smith",
        "title": "Director of Toxicology",
        "company": "Pfizer",
        "location": "Boston, MA",
        "email": None,
        "phone": None,
        "source": "linkedin",
        "tenure_months": 24
    })
    leads.append({
        "name": "Dr. John Doe",
        "title": "VP Preclinical",
        "company": "Novartis",
        "location": "Basel, Switzerland",
        "email": None,
        "phone": None,
        "source": "linkedin",
        "tenure_months": 12
    })
    return leads

def scrape_publications(publication_keywords, job_titles, max_results=20):
    leads = []
    query = " AND ".join(publication_keywords)
    handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
    record = Entrez.read(handle)
    if record["IdList"]:
        fetch = Entrez.efetch(db="pubmed", id=",".join(record["IdList"]), rettype="medline", retmode="text")
        articles = fetch.read().split("\n\n")
        for article in articles:
            author = re.search(r"AU  - (.+)", article)
            title = re.search(r"TI  - (.+)", article)
            affiliation = re.search(r"AD  - (.+)", article)
            pub_date = re.search(r"DP  - (\d{4})", article)
            if author and title:
                aff_text = affiliation.group(1) if affiliation else ""
                location_match = re.search(r',\s*([^,]+,\s*[A-Z]{2}|[^,]+,\s*[^,]+)$', aff_text)
                location = location_match.group(1) if location_match else ""
                if any(jt.lower() in aff_text.lower() for jt in job_titles):
                    recency_score = 0
                    if pub_date:
                        year = int(pub_date.group(1))
                        if datetime.now().year - year <= 2:
                            recency_score = 10
                    leads.append({
                        "name": author.group(1),
                        "title": next((jt for jt in job_titles if jt.lower() in aff_text.lower()), "Researcher"),
                        "company": aff_text,
                        "location": location,
                        "email": None,
                        "phone": None,
                        "source": "pubmed",
                        "publication": title.group(1),
                        "recency_score": recency_score
                    })
    return leads


def scrape_conferences(conference_urls, job_titles):
    leads = []
    for url in conference_urls:

        leads.append({
            "name": "Dr. Alice Brown",
            "title": "Head of Safety Assessment",
            "company": "Gilead",
            "location": "San Francisco, CA",
            "email": None,
            "phone": None,
            "source": "conference",
            "publication": "AACR 2024 Speaker",
        })
    return leads

def enrich_budget_info(leads):

    for lead in leads:
        if "Pfizer" in lead.get("company", ""):
            lead["funding_score"] = 20
        else:
            lead["funding_score"] = 10
    return leads


def enrich_contacts(leads):
    for lead in leads:
        if lead.get("company"):

            lead["email"] = f"{lead['name'].split()[0].lower()}@{lead['company'].split()[0].lower()}.com"
            lead["phone"] = "555-123-4567"
    return leads


def rank_leads(leads):
    for lead in leads:
        score = 0

        if lead["title"]:
            if "director" in lead["title"].lower(): score += 30
            elif "vp" in lead["title"].lower(): score += 25
            elif "head" in lead["title"].lower(): score += 20

        if lead.get("email"): score += 25

        if lead.get("source") == "pubmed": score += 25

        score += lead.get("recency_score", 0)

        if lead.get("source") == "conference": score += 15

        score += lead.get("funding_score", 0)

        if lead.get("location"): score += 10
        lead["score"] = score
    return sorted(leads, key=lambda x: x["score"], reverse=True)


def display_scoreboard(leads):
    st.set_page_config(layout="wide")
    st.title("High-Intent Lead Scoreboard")
    df = pd.DataFrame(leads)
    df = df[["name","title","company","location","email","phone","source","score"]].sort_values("score", ascending=False)
    st.dataframe(df, use_container_width=True, hide_index=True)
    st.download_button(
        "Download CSV",
        df.to_csv(index=False),
        "lead_scoreboard.csv",
        "text/csv"
    )

def run_pipeline():
    job_titles = ["Director", "VP", "Head"]
    publication_keywords = ["Drug-Induced Liver Injury", "Organ-on-chip", "Hepatic spheroids"]
    conference_urls = ["https://example-conference.org/speakers"]

    leads = []
    leads += scrape_social_networks(job_titles)
    leads += scrape_publications(publication_keywords, job_titles)
    leads += scrape_conferences(conference_urls, job_titles)
    leads = enrich_budget_info(leads)
    leads = enrich_contacts(leads)
    leads = rank_leads(leads)
    return leads

ranked_leads = run_pipeline()
display_scoreboard(ranked_leads)


2025-12-16 15:41:58.888 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 15:41:58.889 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 15:41:59.094 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-12-16 15:41:59.095 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 15:41:59.097 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 15:41:59.121 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='c

In [9]:
!streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.50.184.81:8501

  Stopping...
  Stopping...


In [10]:
!pip install streamlit pyngrok


In [13]:
from pyngrok import ngrok
import subprocess

process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)


public_url = ngrok.connect(8501)
print("Streamlit URL:", public_url)


ERROR:pyngrok.process.ngrok:t=2025-12-16T16:03:52+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-12-16T16:03:52+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-12-16T16:03:52+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [14]:
!apt-get install cloudflared


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package cloudflared


In [15]:
!streamlit run app.py --server.port 8501 &




2025-12-16 16:05:17.249 Port 8501 is already in use


In [16]:
!cloudflared tunnel --url http://localhost:8501


/bin/bash: line 1: cloudflared: command not found
